# <center><b>Data Claiming</b></center>

<h2>Preparation</h2>

In [1]:
# include libraries
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import urllib.robotparser
import re
import requests
import requests_cache
import time, json
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# target link
anchor_user = "one direction"
target_web_link = 'https://soundcloud.com'
test_cache_web_link1 = 'https://raw.githubusercontent.com/KienTrann/requests-cache-testing/main/should_be_cached.txt'
test_cache_web_link2 = 'https://raw.githubusercontent.com/KienTrann/requests-cache-testing/main/should_not_be_cached.txt'
chrome_driver_binary = "D:\Sequence\Comp-Suppoter\Chrome\chromedriver"

In [2]:
# get the website source
session = HTMLSession()
source = session.get('https://thinhsuy.github.io/VisualPage/#')
tag = source.html.find('.menu-a', first=True)
print(f"Tag's HTML string: {tag.html}")
print(f"Tag's text string: {tag.text}")
print(f"Attributes dictionary tag: {tag.attrs}")

Tag's HTML string: <a class="menu-a" href="#">About</a>
Tag's text string: About
Attributes dictionary tag: {'class': ('menu-a',), 'href': '#'}


In [3]:
# check the robot.txt of website
rp = urllib.robotparser.RobotFileParser()
rp.set_url(target_web_link+'/robots.txt')
rp.read()
rp.can_fetch('*', target_web_link)

True

In [4]:
# Regular Expression
re.findall(r'\w{5}', tag.text)

['About']

In [5]:
# Working with dynamic webpage
session = HTMLSession()
source = session.get(target_web_link)
try: source.html.render()
except: print('Not running .render() in Jupyter! Run it in powershell file.py instead')

Not running .render() in Jupyter! Run it in powershell file.py instead


<h2>Hand on crawling data</h2>

In [6]:
# Limit the total step that the request would crawl down data
def check_filter(response: requests.models.Response)->bool:
    return True if '"incomplete_result":true' in response.text else False

requests_cache.install_cache("my_cache", filter_fn=check_filter)

# Request function
sleep_time = 1
def get_request(url):
    r = requests.get(url)
    while not(r.ok):
        time.sleep(sleep_time)
        r = requests.get(url)
    return r

# Testing
print(f"Test from link 1: \n{get_request(test_cache_web_link1).text}")
print(f"Test from link 2: \n{get_request(test_cache_web_link2).text}")

Test from link 1: 
abc
xyz
"incomplete_results":false

Test from link 2: 
abc
xyz
"incomplete_results":true



In [7]:
def OpenWebBrowser(driver, link, by=By.ID, id_coockie_trust="none", delay_element_loading=10):
    time.sleep(sleep_time)
    driver.get(link)
    if id_coockie_trust=="none": return True
    try:
        WebDriverWait(driver, delay_element_loading).until(
            EC.presence_of_element_located(
                (by, id_coockie_trust)
            ))
    except TimeoutException: 
        print("Runtime out")
        return False
    time.sleep(sleep_time)
    driver.find_element(by, id_coockie_trust).click()
    return True

def ScrollScreen(driver, epochs=1, time_sleep=0.5):
    for _ in range(epochs):
        time.sleep(time_sleep)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [34]:
def GetRelatedUsers(driver):
    def GetUserInforFromMainPage(page_url):
        driver.get(page_url)
        time.sleep(sleep_time)
        stats_value = driver.find_elements(By.CLASS_NAME, "infoStats__value")
        time.sleep(sleep_time)
        values = [0,0,0]
        for i in range(3):
            try: values[i] = stats_value[i].text
            except: break
        time.sleep(sleep_time)
        lasts_value = driver.find_elements(By.CLASS_NAME,'sc-visuallyhidden')
        time.sleep(sleep_time)
        df_dic = {'Followers': values[0], 'Following': values[1], 'Tracks': values[2]}
        for i in range(2):
            try: values[i] = lasts_value[i].text
            except: values[i] = 0
        df_dic['Last_action'] = values[0]
        df_dic['Last_track_played'] = values[1]
        return df_dic 

    headers = ["Name", "Followers", "Following", "Tracks", "Last_action", "Last_track_played"]
    user_df = pd.DataFrame(columns=headers)
    user_urls = driver.find_elements(By.CSS_SELECTOR, ".sc-link-dark.sc-link-primary")
    time.sleep(0.5)
    chosen_url = [value.get_attribute('href') for value in user_urls if value.get_attribute('href').count('/')<=3][0]
    user_dict = GetUserInforFromMainPage(chosen_url)
    user_dict['Name'] = anchor_user
    user_df = user_df.append(user_dict, ignore_index=True)
    return user_df

In [35]:
driver = webdriver.Chrome()
url = target_web_link + '/search/people?q=' + anchor_user
if (OpenWebBrowser(driver, url, id_coockie_trust="onetrust-accept-btn-handler")):
    #ScrollScreen(driver, epochs=1)
    df = GetRelatedUsers(driver)
driver.close()
df

C:\Users\Thinh Nguyen\AppData\Local\Temp\ipykernel_10260\4001606070.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  user_df = user_df.append(user_dict, ignore_index=True)


,Name,Followers,Following,Tracks,Last_action,Last_track_played
0,one direction,3.01M,0,129,Posted 7 years ago,"2,214,463 plays"
